# Community Deception

Connect Google Drive to access the dataset.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## Install Pytorch Geometric

If we are on Kaggle we need to run the following cells to install Pytorch Geometric

In [ ]:
import torch
import os

os.environ["TORCH"] = torch.__version__

# On Colab we can have TORCH+CUDA on os.environ["TORCH"]

# Check if there is the cuda version on TORCH
if torch.cuda.is_available():
    print("CUDA is available")
    print(torch.version.cuda)
    if "+" not in os.environ["TORCH"]:
        os.environ["TORCH"] += "+cu" + \
            torch.version.cuda.replace(".", "")

print(os.environ["TORCH"])

Install torch geometric and optional dependencies:

In [ ]:
# ! pip install torch_geometric
# Optional dependencies:
# # ! pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html
# # ! pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
# ! pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-${TORCH}.html

or

In [ ]:
# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.0.0


Install Graph Library:

In [ ]:
# Graph
! pip install igraph
! pip install cdlib[C]

**IMPORTANT!!!**
After the libraries installation, restart the runtime and start executing the cells below

## Import Libraries

In [1]:
# Import torch and os another time to reset the colab enviroment after PyG installation
import torch
import os
import gc

# Typing
from typing import List, Tuple, Set
from collections import Counter, namedtuple

# Deep Learning
from torch_geometric.utils import from_networkx
from torch_geometric.data import Data
from torch_geometric.data import Batch
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.nn import global_mean_pool
from torch.distributions import MultivariateNormal

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import scipy

# Graph
from cdlib import algorithms
import cdlib
import networkx as nx
import igraph as ig


# Misc
from enum import Enum
from tqdm import trange
import math
import random
import json

# Plot
import matplotlib.pyplot as plt
plt.style.use('default')

ModuleNotFoundError: No module named 'torch_geometric'

## Utils

In [ ]:
class FilePaths(Enum):
    """Class to store file paths for data and models"""
    # ° Local
    # DATASETS_DIR = 'dataset/data'
    # LOG_DIR    = 'src/logs/'
    # TEST_DIR = 'test/'
    # ° Kaggle
    # DATASETS_DIR = '/kaggle/input/network-community'
    # LOG_DIR = '/kaggle/working/logs/'
    # TEST_DIR = '/kaggle/working/test/'
    # ° Google Colab
    DATASETS_DIR = "/content/drive/MyDrive/Sapienza/Tesi/Datasets"
    LOG_DIR = "/content/drive/MyDrive/Sapienza/Tesi/Logs/"
    TEST_DIR = "/content/drive/MyDrive/Sapienza/Tesi/Test/
    
    # Dataset file paths
    KAR = DATASETS_DIR + '/kar.mtx'
    DOL = DATASETS_DIR + '/dol.mtx'
    MAD = DATASETS_DIR + '/mad.mtx'
    LESM = DATASETS_DIR + '/lesm.mtx'
    POLB = DATASETS_DIR + '/polb.mtx'
    WORDS = DATASETS_DIR + '/words.mtx'
    ERDOS = DATASETS_DIR + '/erdos.mtx'
    POW = DATASETS_DIR + '/pow.mtx'
    FB_75 = DATASETS_DIR + '/fb-75.mtx'
    DBLP = DATASETS_DIR + '/dblp.mtx'
    ASTR = DATASETS_DIR + '/astr.mtx'
    AMZ = DATASETS_DIR + '/amz.mtx'
    YOU = DATASETS_DIR + '/you.mtx'
    ORK = DATASETS_DIR + '/ork.mtx'


class HyperParams(Enum):
    """Hyperparameters for the Environment"""
    # Numeber of possible action with BETA=30, is 30% of the edges
    BETA = 10  
    # Weight to balance the reward
    WEIGHT = 0.1  # 0.001, 0.01, 0.1, 1, 10
    
    """ Graph Encoder Parameters """""
    STATE_DIM = 64
    # G_HIDDEN_SIZE_1 = 128
    # G_HIDDEN_SIZE_2 = 64
    # G_EMBEDDING_SIZE = 32

    """ Agent Parameters"""
    HIDDEN_SIZE_1 = 32
    HIDDEN_SIZE_2 = 32
    ACTION_DIM = 1      # We will return a  N*1 vector of actions, where N is the number of nodes
    # ACTION_STD = 0.5
    EPS_CLIP = np.finfo(np.float32).eps.item()  # 0.2
    LR = 0.0001
    GAMMA = 0.5 # 0.97
    BEST_REWARD = 0.7  # -np.inf

    """ Training Parameters """
    # Number of episodes to collect experience
    MAX_EPISODES = 1000  # 200 # 15000
    # Dictonary for logging
    LOG_DICT = {
        'train_reward': [],
        # Number of steps per episode
        'train_steps': [],
        # Average reward per step
        'train_avg_reward': [],
        # Average Actor loss per episode
        'a_loss': [],
        # Average Critic loss per episode
        'v_loss': [],
        # set max number of training episodes
        'train_episodes': MAX_EPISODES,
    }
    
    """Graph Generation Parameters"""
    N_NODE = 10000
    TAU1 = 3
    TAU2 = 1.5
    MU = 0.1             # TODO: Test also 0.3 and 0.6
    AVERAGE_DEGREE = 5
    MIN_COMMUNITY = 20
    SEED= 10

    """Old Training Parameters"""
    # Maximum number of time steps per episode
    # MAX_TIMESTEPS = 10  # ! Unused, I set it to the double of the edge budget
    # Update the policy after N timesteps
    # UPDATE_TIMESTEP = 100  # ! Unused, I set it to 10 times the edge budget
    # Update policy for K epochs
    # K_EPOCHS = 20
    # Print info about the model after N episodes
    # LOG_INTERVAL = 20
    # Exit if the average reward is greater than this value
    # SOLVED_REWARD = 0.7
    # Save model after N episodes
    # SAVE_MODEL = int(MAX_EPISODES / 10)
    # Use a random seed
    # RANDOM_SEED = 42


class DetectionAlgorithms(Enum):
    """
    Enum class for the detection algorithms
    """
    LOUV = "louvain"
    WALK = "walktrap"
    GRE = "greedy"
    INF = "infomap"
    LAB = "label_propagation"
    EIG = "eigenvector"
    BTW = "edge_betweenness"
    SPIN = "spinglass"
    OPT = "optimal"
    SCD = "scalable_community_detection"


class Utils:
    """Class to store utility functions"""

    @staticmethod
    def import_mtx_graph(file_path: str) -> nx.Graph:
        """
        Import a graph from a .mtx file

        Parameters
        ----------
        file_path : str
            File path of the .mtx file

        Returns
        -------
        nx.Graph
            Graph imported from the .mtx file
        """
        try:
            graph_matrix = scipy.io.mmread(file_path)
            graph = nx.Graph(graph_matrix)
            for node in graph.nodes:
                # graph.nodes[node]['name'] = node
                graph.nodes[node]['num_neighbors'] = len(
                    list(graph.neighbors(node)))
            return graph
        except Exception as exception:
            print("Error: ", exception)
            return None
    
    @staticmethod
    def generate_lfr_benchmark_graph(
        n: int=HyperParams.N_NODE.value,
        tau1: float=HyperParams.TAU1.value,
        tau2: float=HyperParams.TAU2.value,
        mu: float=HyperParams.MU.value,              
        average_degree: float=HyperParams.AVERAGE_DEGREE.value, 
        min_community: int=HyperParams.MIN_COMMUNITY.value, 
        seed: int=HyperParams.SEED.value)->Tuple[nx.Graph, str]:
        """
        Generate a LFR benchmark graph for community detection algorithms.

        Parameters
        ----------
        n : int, optional
            _description_, by default 250
        tau1 : float, optional
            _description_, by default 3
        tau2 : float, optional
            _description_, by default 1.5
        mu : float, optional
            _description_, by default 0.1
        average_degree : float, optional
            _description_, by default 5
        min_community : int, optional
            _description_, by default 20
        seed : int, optional
            _description_, by default 10

        Returns
        -------
        nx.Graph
            Synthetic graph generated with the LFR benchmark
        file_path : str
            Path to the file where the graph is saved
        """
        graph = nx.generators.community.LFR_benchmark_graph(
            n=n,
            tau1=tau1,
            tau2=tau2,
            mu=mu,
            average_degree=average_degree,
            min_community=min_community,
            seed=seed)
        file_path = FilePaths.DATASETS_DIR.value + f"/lfr_benchmark_mu-{mu}.mtx"
        nx.write_edgelist(graph, file_path, data=False)
        # Delete community attribute from the nodes to handle PyG compatibility
        for node in graph.nodes:
            if 'community' in graph.nodes[node]:
                del graph.nodes[node]['community']
        for edge in graph.edges:
            graph.edges[edge]['weight'] = 1
        return graph, file_path
        
    @staticmethod
    def check_dir(path: str):
        """
        Check if the directory exists, if not create it.

        Parameters
        ----------
        path : str
            Path to the directory
        """
        if not os.path.exists(path):
            os.makedirs(path)
    
    @staticmethod
    def plot_training(
        log: dict, 
        env_name: str, 
        detection_algorithm: str,
        file_path: str,
        window_size: int=100):
        """Plot the training results

        Parameters
        ----------
        log : dict
            Dictionary containing the training logs
        env_name : str
            Name of the environment
        detection_algorithm : str
            Name of the detection algorithm
        file_path : str
            Path to save the plot
        window_size : int, optional
            Size of the rolling window, by default 100
        """
        def plot_time_series(
            list_1: List[float],
            list_2: List[float],
            label_1: str,
            label_2: str,
            color_1: str,
            color_2: str,
            file_name: str):
            _, ax1 = plt.subplots()
            color = 'tab:'+color_1
            ax1.set_xlabel("Episode")
            ax1.set_ylabel(label_1, color=color)
            ax1.plot(list_1, color=color)
            ax1.tick_params(axis='y', labelcolor=color)

            ax2 = ax1.twinx()
            color = 'tab:'+color_2
            ax2.set_ylabel(label_2, color=color)
            ax2.plot(list_2, color=color)
            ax2.tick_params(axis='y', labelcolor=color)

            plt.title(
                f"Training on {env_name} graph with {detection_algorithm} algorithm")
            plt.savefig(file_name)
            plt.show()
        
        def plot_rolling_window(
            list_1: List[float],
            list_2: List[float],
            label_1: str,
            label_2: str,
            file_name: str,
            window_size: int = 100):
            time_series_1 = np.array(list_1)
            time_series_2 = np.array(list_2)
            # Compute the rolling windows of the time series data using NumPy
            rolling_data_1 = np.convolve(time_series_1, np.ones(
                window_size) / window_size, mode='valid')
            rolling_data_2 = np.convolve(time_series_2, np.ones(
                window_size) / window_size, mode='valid')
            # Plot the rolling windows of the time series data using matplotlib
            plt.plot(rolling_data_1, label=label_1)
            plt.plot(rolling_data_2, label=label_2)
            plt.title("Rolling Window")
            plt.xlabel("Epochs")
            # plt.ylabel("Epochs")
            plt.legend()
            plt.savefig(file_name)
            plt.show()
        
        file_path = file_path+"/"+env_name+"_"+detection_algorithm
        plot_time_series(
            log['train_avg_reward'],
            log['train_steps'],
            'Avg Reward',
            'Steps per Epoch',
            'blue',
            'orange',
            file_path+"_training_reward.png",
        )
        plot_time_series(
            log["a_loss"],
            log["v_loss"],
            'Actor Loss',
            'Critic Loss',
            'green',
            'red',
            file_path+"_training_loss.png",
        )

        # Same plot with rolling window
        plot_rolling_window(
            log['train_reward'], 
            log['train_steps'], 
            'Avg Reward', 
            'Steps per Epoch',
            file_path+"_rolling_training_reward.png"
        )
        plot_rolling_window(
            log["a_loss"],
            log["v_loss"],
            'Actor Loss',
            'Critic Loss',
            file_path+"_rolling_training_loss.png"
        )
    
    @staticmethod
    def save_training(
            log: dict,
            env_name: str,
            detection_algorithm: str,
            file_path: str):
        """Plot the training results

        Parameters
        ----------
        log : dict
            Dictionary containing the training logs
        env_name : str
            Name of the environment
        detection_algorithm : str
            Name of the detection algorithm
        file_path : str
            Path to save the plot
        """
        file_name = f"{file_path}/{env_name}_{detection_algorithm}_results.json"
        with open(file_name, "w", encoding="utf-8") as f:
            json.dump(log, f, indent=4)


In [ ]:
# Create paths
Utils.check_dir(FilePaths.LOG_DIR.value)
Utils.check_dir(FilePaths.TEST_DIR.value)

## Community Algorithms

### Communities Detection

In [ ]:
class CommunityDetectionAlgorithm(object):
    """Class for the community detection algorithms using CDLIB"""
    def __init__(self, alg_name: str) -> None:
        """
        Initialize the DetectionAlgorithm object
        
        Parameters
        ----------
        alg_name : str
            The name of the algorithm
        """
        self.alg_name = alg_name
    
    def compute_community(self, graph: nx.Graph) -> cdlib.NodeClustering:
        """Compute the community partition of the graph

        Parameters
        ----------
        graph : nx.Graph
            Input graph

        Returns
        -------
        cdlib.NodeClustering
            Cdlib NodeClustering object
        """
        # Rename DetectionAlgorithms Enum to da for convenience
        da = DetectionAlgorithms
        # Choose the algorithm
        if self.alg_name == da.LOUV.value:
            return algorithms.louvain(graph)
        elif self.alg_name == da.WALK.value:
            return algorithms.walktrap(graph)
        elif self.alg_name == da.GRE.value:
            return algorithms.greedy_modularity(graph)
        elif self.alg_name == da.INF.value:
            return algorithms.infomap(graph)
        # elif self.alg_name == da.LAB.value:
        #    # ! Return a EdgeClustering object
        #    return algorithms.label_propagation(graph)
        elif self.alg_name == da.EIG.value:
            return algorithms.eigenvector(graph)
        # elif self.alg_name == da.BTW.value:
        #     return self.compute_btw(graph, args)
        elif self.alg_name == da.SPIN.value:
            return algorithms.spinglass(graph)
        # elif self.alg_name == da.OPT.value:
        #    return self.compute_opt(graph, args)
        # elif self.alg_name == da.SCD.value:
        #    return self.compute_scd(graph)
        else:
            raise ValueError('Invalid algorithm name')

### Normalized Mutual Information Score

In [ ]:

class NormalizedMutualInformation(object):
    @staticmethod
    def calculate_confusion_matrix(
            communities_old: List[List[int]],
            communities_new: List[List[int]]) -> Counter:
        """
        Calculate the confusion matrix between two sets of communities.
        Where the element (i, j) of the confusion matrix is the number of shared
        members between an initially detected community C_i and the community
        C_j after deception.

        Parameters
        ----------
        communities_old : List[List[int]]
            Communities before deception
        communities_new : List[List[int]]
            Communities after deception

        Returns
        -------
        confusion_matrix : Counter
            Confusion matrix
        """
        confusion_matrix = Counter()
        #° Avoid to process the same community twice
        #BUG ZeroDivisionError if we use this optimization
        #BUG processed_new = set()
        for i, old in enumerate(communities_old):
            for j, new in enumerate(communities_new):
                #BUG if j not in processed_new:
                intersection = len(set(old) & set(new))
                confusion_matrix[(i, j)] = intersection
                #BUG    if intersection > 0:
                #BUG        processed_new.add(j)
        return confusion_matrix

    @staticmethod
    def calculate_sums(confusion_matrix: Counter) -> Tuple[Counter, Counter, int]:
        """
        Calculate the row sums, column sums and total sum of a confusion matrix.

        Parameters
        ----------
        confusion_matrix : Counter
            Confusion matrix

        Returns
        -------
        (row_sums, col_sums, total_sum) : Tuple[Counter, Counter, int]
            Tuple containing the row sums, column sums and total sum of the
            confusion matrix.
        """
        row_sums = Counter()
        col_sums = Counter()
        total_sum = 0
        for (i, j), value in confusion_matrix.items():
            row_sums[i] += value
            col_sums[j] += value
            total_sum += value
        return row_sums, col_sums, total_sum

    def compute_nmi(
            self,
            communities_old: List[List[int]],
            communities_new: List[List[int]]) -> float:
        """
        Calculate the normalized mutual information between two sets of
        Communities.

        Parameters
        ----------
        communities_old : List[List[int]]
            List of communities before deception
        communities_new : List[List[int]]
            List of communities after deception

        Returns
        -------
        nmi : float
            Normalized mutual information, value between 0 and 1.
        """
        confusion_matrix = self.calculate_confusion_matrix(
            communities_old, communities_new)
        row_sums, col_sums, total_sum = self.calculate_sums(confusion_matrix)

        # Numerator
        nmi_numerator = 0
        for (i, j), n_ij in confusion_matrix.items():
            n_i = row_sums[i]
            n_j = col_sums[j]
            try:
                nmi_numerator += n_ij * math.log((n_ij * total_sum) / (n_i * n_j))
            except ValueError:
                # We could get a math domain error if n_ij is 0
                continue

        # Denominator
        nmi_denominator = 0
        for i, n_i in row_sums.items():
            nmi_denominator += n_i * math.log(n_i / total_sum)
        for j, n_j in col_sums.items():
            nmi_denominator += n_j * math.log(n_j / total_sum)
        # Normalized mutual information
        nmi_score = -2 * nmi_numerator / nmi_denominator
        return nmi_score

## Enviroment

In [2]:
class GraphEnvironment(object):
    """Enviroment where the agent will act, it will be a graph with a community"""

    def __init__(
        self, 
        graph: nx.Graph, 
        community: List[int], 
        idx_community: int,
        node_target: int,
        env_name: str,
        community_detection_algorithm: str,
        beta: float = HyperParams.BETA.value, 
        weight: float = HyperParams.WEIGHT.value) -> None:
        """Constructor for Graph Environment
        Parameters
        ----------
        graph : nx.Graph
            Graph to use for the environment
        community : List[int]
            Community of node we want to remove from it
        idx_community : int
            Index of the community in the list of communities
        nodes_target : int
            Node we want to remove from the community
        env_name : str
            Name of the environment, i.e. name of the dataset
        community_detection_algorithm : str
            Name of the community detection algorithm to use
        beta : float, optional
            Percentage of edges to remove, by default HyperParams.BETA.value
        weight : float, optional
            Weight of the metric, by default HyperParams.WEIGHT.value
        """
        self.graph = graph
        self.graph_copy = graph.copy()
        # Get the Number of connected components
        self.n_connected_components = nx.number_connected_components(graph)
        
        # Community to hide
        self.community_target = community
        self.idx_community_target = idx_community
        
        # Node to remove from the community
        assert node_target in community, "Node must be in the community"
        self.node_target = node_target
        
        assert beta >= 0 and beta <= 100, "Beta must be between 0 and 100"
        self.beta = beta
        self.weight = weight
        self.env_name = env_name
        
        # Community Algorithms objects
        self.detection = CommunityDetectionAlgorithm(community_detection_algorithm)
        self.deception = DeceptionScore(self.community_target)
        # self.safeness = Safeness(self.graph, self.community_target, self.node_target)
        self.nmi = NormalizedMutualInformation()
        # Compute the community structure of the graph, before the action,
        # i.e. before the deception
        self.community_structure_start = self.detection.compute_community(graph)
        # ! It is a NodeClustering object
        self.community_structure_old = self.community_structure_start
        
        # Compute the edge budget for the graph
        self.edge_budget = self.get_edge_budget()
        # Amount of budget used
        self.used_edge_budget = 0
        # Whether the budget for the graph rewiring is exhausted, or the target
        # node does not belong to the community anymore
        self.stop_episode = False
        self.rewards = 0
        # Reward of the previous step
        self.old_rewards = 0
        
        # Compute the set of possible actions
        self.possible_actions = self.get_possible_actions()
        # Length of the list of possible actions to add
        self.len_add_actions = len(self.possible_actions["ADD"])
        
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def change_target_node(self, node_target: int=None) -> None:
        """
        Change the target node to remove from the community

        Parameters
        ----------
        node_target : int, optional
            Node to remove from the community, by default None
        """
        if node_target is None:
            # Choose a node randomly from the community
            idx_node = random.randint(0, len(self.community_target)-1)
            self.node_target = self.community_target[idx_node]
        else:
            self.node_target = node_target
    
    def change_target_community(
        self, 
        community: List[int]=None, 
        idx_community: int=None,
        node_target: int=None) -> None:
        """
        Change the target community from which we want to hide the node

        Parameters
        ----------
        community : List[int]
            Community of node we want to remove from it
        idx_community : int
            Index of the community in the list of communities
        """
        if community is None:
            # Choose a community randomly from the list of communities
            self.idx_community_target = random.randint(
                0, len(self.community_structure_start.communities)-1)
            self.community_target = self.community_structure_start.communities[
                self.idx_community_target]
        else:
            self.community_target = community
            self.idx_community_target = idx_community
        self.change_target_node(node_target=node_target)
    
    def get_community_target_idx(
        self,
        community_structure: List[List[int]],
        community_target: List[int]) -> int:
        """
        Returns the index of the target community in the list of communities
        As the target community after a rewiring action we consider the community
        with the highest number of nodes equal to the initial community.
        
        Parameters
        ----------
        community_structure : List[List[int]]
            List of communities
        community_target : List[int]
            Community of node we want to remove from it
        
        Returns
        -------
        max_list_idx : int
            Index of the target community in the list of communities
        """
        max_count = 0
        max_list_idx = 0
        for i, lst in enumerate(community_structure):
            count = sum(1 for x in lst if x in community_target)
            if count > max_count:
                max_count = count
                max_list_idx = i
        return max_list_idx
    
    def get_edge_budget(self) -> int:
        """
        Computes the edge budget for each graph

        Returns
        -------
        int
            Edge budgets of the graph
        """
        return int(math.ceil((self.graph.number_of_edges() * self.beta / 100)))

    def get_reward(self, metric: float) -> Tuple[float, bool]:
        """
        Computes the reward for the agent
        
        Parameters
        ----------
        metric : float
            Metric to use to compute the reward

        Returns
        -------
        reward : float
            Reward of the agent
        done : bool
            Whether the episode is finished, if the target node does not belong
            to the community anymore, the episode is finished
        """
        # if the target node still belongs to the community, the reward is negative 
        communities_list = self.community_structure_new.communities
        if self.node_target in communities_list[self.idx_community_target]:
            reward = -self.weight * metric
            return reward, False
        # if the target node does not belong to the community anymore, the reward is positive
        reward = 1 - (self.weight * metric)
        return reward, True

    def reset(self) -> Data:
        """
        Reset the environment

        Returns
        -------
        adj_matrix : torch.Tensor
            Adjacency matrix of the graph
        """
        self.used_edge_budget = 0
        self.stop_episode = False
        self.graph = self.graph_copy.copy()
        self.possible_actions = self.get_possible_actions()
        
        # Return a PyG Data object
        self.data_pyg = from_networkx(self.graph)
        # Initialize the node features
        self.data_pyg.x = torch.randn([self.data_pyg.num_nodes, HyperParams.G_IN_SIZE.value])
        # Initialize the batch
        self.data_pyg.batch = torch.zeros(self.data_pyg.num_nodes).long()
        return self.data_pyg.to(self.device)
    
    def step(self, action: int) -> Tuple[Data, float]:
        """
        Step function for the environment
        
        Parameters
        ----------
        action : int
            Integer representing a node in the graph, it will be the destination
            node of the rewiring action (out source node is always the target node).
            
        Returns
        -------
        self.graph, self.rewards: Tuple[torch.Tensor, float]
            Tuple containing the new graph and the reward 
        """
        # ° ---- ACTION ---- ° #
        # Take action, budget_consumed can be 0 or 1, i.e. if the action has
        # been applied or not
        budget_consumed = self.apply_action(action)
        # Set a negative reward if the action has not been applied
        if budget_consumed == 0:
            self.rewards = -2
            # The state is the same as before
            return self.data_pyg.to(self.device), self.rewards, self.stop_episode
        
        # ° ---- METRICS ---- ° #
        # Compute the new Community Structure after the action
        self.community_structure_new = self.detection.compute_community(self.graph)
        # Search the index of the target community in the new list of communities
        self.idx_community_target = self.get_community_target_idx(
            self.community_structure_new.communities, 
            self.community_target)
        
        # ! It is a NodeClustering object
        # nmi = self.community_structure_new.normalized_mutual_information(
        #    self.community_structure_old).score
        # NOTE: My implementation of NMI is faster than the one in cdlib
        # Normalized Mutual Information, value between 0 and 1
        nmi = self.nmi.compute_nmi(
            self.community_structure_old.communities, 
            self.community_structure_new.communities)
        
        # Deception Score, value between 0 and 1
        # deception_score = self.deception.compute_deception_score(self.community_structure_new.communities, self.n_connected_components)
        # Safeness, value between 0 and 1
        # node_safeness = self.safeness.compute_community_safeness(self.nodes_target)
        # node_safeness = self.safeness.compute_node_safeness(self.nodes_target[0]) # ! Assume that there is only one node to hide
        
        self.community_structure_old = self.community_structure_new
        
        # ° ---- REWARD ---- ° #
        self.rewards, done = self.get_reward(nmi)
        # If the target node does not belong to the community anymore, 
        # the episode is finished
        if done:
            self.stop_episode = True
        
        # ° ---- BUDGET ---- ° #
        # Compute the remaining budget
        remaining_budget = self.edge_budget - self.used_edge_budget
        # Decrease the remaining budget
        updated_budget = remaining_budget - budget_consumed
        # Update the used edge budget
        self.used_edge_budget += (remaining_budget - updated_budget)
        # If the budget for the graph rewiring is exhausted, stop the episode
        if remaining_budget < 1:
            self.stop_episode = True
            # If the budget is exhausted, and the target node still belongs to
            # the community, the reward is negative
            if not done:
                self.rewards = -1

        # ° ---- PyG Data ---- ° #
        # TEST: Avoid to use from_networkx
        edge_list = nx.to_edgelist(self.graph)
        # remove weights
        edge_list = [[e[0], e[1]] for e in edge_list]
        edge_list += [[e[1], e[0]] for e in edge_list]
        # order the list, first by first element, then by second element
        edge_list = sorted(edge_list, key=lambda x: (x[0], x[1]))
        # Create tensor
        edge_list = torch.tensor(edge_list)
        edge_list_t = torch.transpose(edge_list, 0, 1)
        del edge_list
        self.data_pyg.edge_index = edge_list_t
        # TEST END
        
        # Return a PyG Data object
        # TEST data = from_networkx(self.graph)
        # Assign the node features and the batch of the old graph to the new graph
        # TEST data.x = self.data_pyg.x
        # TEST data.batch = self.data_pyg.batch
        # Update the old graph pyg data object
        # TEST self.data_pyg = data
        return self.data_pyg.to(self.device), self.rewards, self.stop_episode
    
    def get_possible_actions(self) -> dict:
        """
        Returns all the possible actions that can be applied to the graph
        given a source node (self.node_target). The possible actions are:
            - Add an edge between the source node and a node outside the community
            - Remove an edge between the source node and a node inside the community
        
        Returns
        -------
        self.possible_actions : dict
            Dictionary containing the possible actions that can be applied to
            the graph. The dictionary has two keys: "ADD" and "REMOVE", each
            key has a list of tuples as value, where each tuple is an action.
        """
        possible_actions = {"ADD": set(), "REMOVE": set()}
        # Helper functions to check if a node is in/out-side the community
        def in_community(node):
            return node in self.community_target

        def out_community(node):
            return node not in self.community_target
        
        u = self.node_target
        for v in self.graph.nodes():
            if u == v:
                continue
            # We can remove an edge iff both nodes are in the community
            if in_community(u) and in_community(v):
                if self.graph.has_edge(u, v):
                    if (v, u) not in possible_actions["REMOVE"]:
                        possible_actions["REMOVE"].add((u, v))
            # We can add an edge iff one node is in the community and the other is not
            elif (in_community(u) and out_community(v)) \
                    or (out_community(u) and in_community(v)):
                # Check if there is already an edge between the two nodes
                if not self.graph.has_edge(u, v):
                    if (v, u) not in possible_actions["ADD"]:
                        possible_actions["ADD"].add((u, v))
        return possible_actions
    
    def apply_action(self, action: int) -> int:
        """
        Applies the action to the graph, if there is an edge between the two 
        nodes, it removes it, otherwise it adds it

        Parameters
        ----------
        action : int
            Integer representing a node in the graph, it will be the destination
            node of the rewiring action (out source node is always the target node).
        
        Returns
        -------
        budget_consumed : int
            Amount of budget consumed, 1 if the action has been applied, 0 otherwise
        """
        action = (self.node_target, action)   
        # We need to take into account both the actions (u,v) and (v,u)
        action_reversed = (action[1], action[0])
        if action in self.possible_actions["ADD"]:
            self.graph.add_edge(*action, weight=1)
            self.possible_actions["ADD"].remove(action)
            return 1
        elif action_reversed in self.possible_actions["ADD"]:
            self.graph.add_edge(*action_reversed, weight=1)
            self.possible_actions["ADD"].remove(action_reversed)
            return 1
        elif action in self.possible_actions["REMOVE"]:
            self.graph.remove_edge(*action)
            self.possible_actions["REMOVE"].remove(action)
            return 1
        elif action_reversed in self.possible_actions["REMOVE"]:
            self.graph.remove_edge(*action_reversed)
            self.possible_actions["REMOVE"].remove(action_reversed)
            return 1
        return 0

    def plot_graph(self) -> None:
        """Plot the graph using matplotlib"""
        import matplotlib.pyplot as plt
        nx.draw(self.graph, with_labels=True)
        plt.show()


NameError: name 'HyperParams' is not defined

## Model

### Encoder

In [ ]:
class GraphEncoder(nn.Module):
    def __init__(
        self,
        in_feature, 
        # embdedding_size,
        num_layers=2):
        super(GraphEncoder, self).__init__()

        self.conv_layers = nn.ModuleList()
        # self.conv_layers.append(GCNConv(in_feature, in_feature))
        self.conv_layers.append(GATConv(in_feature, in_feature))
        for _ in range(num_layers - 1):
            # self.conv_layers.append(GCNConv(in_feature, in_feature))
            self.conv_layers.append(GATConv(in_feature, in_feature))
        
        self.relu = nn.LeakyReLU()
        #self.relu = nn.ReLU()
    
    #NOTE Torch Geometric MessagePassing, it takes as input the edge list 
    def forward(self, graph: Data)-> torch.Tensor:
        x, edge_index, batch = graph.x, graph.edge_index, graph.batch

        for conv in self.conv_layers:
            x = conv(x, edge_index)
            x = self.relu(x)
        # embedding = global_mean_pool(x, batch)
        # self.is_nan(x, "x")
        embedding = x + graph.x
        
        return embedding

    def is_nan(self, x, label):
        """Debugging function to check if there are NaN values in the tensor"""
        if torch.isnan(x).any():
            print(label, ":", x)
            raise ValueError(label, "is NaN")


### Network

#### Actor

In [ ]:
class ActorNetwork(nn.Module):
    """Actor Network"""
    
    def __init__(
            self,
            state_dim: int,
            hidden_size_1: int,
            hidden_size_2: int,
            action_dim: int):
        super(ActorNetwork, self).__init__()
        
        self.graph_encoder = GraphEncoder(state_dim)
        self.linear1 = nn.Linear(state_dim, hidden_size_1)
        self.linear2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.linear3 = nn.Linear(hidden_size_2, action_dim)
        
        self.relu = nn.LeakyReLU()
        # self.relu = nn.ReLU()
        # self.tanh = nn.Tanh()

    def forward(self, state: Data):
        embedding = self.graph_encoder(state)
        actions = self.relu(self.linear1(embedding))
        actions = self.relu(self.linear2(actions))
        actions = self.linear3(actions)
        return actions
    
    def is_nan(self, x, label):
        """Debugging function to check if there are NaN values in the tensor"""
        if torch.isnan(x).any():
            print(label, ":", x)
            raise ValueError(label, "is NaN")

#### Critic

In [ ]:
class CriticNetwork(nn.Module):
    def __init__(
        self,
        state_dim: int,
        hidden_size_1: int,
        hidden_size_2: int):
        super(CriticNetwork, self).__init__()
        
        self.graph_encoder = GraphEncoder(state_dim)
        self.linear1 = nn.Linear(state_dim, hidden_size_1)
        self.linear2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.linear3 = nn.Linear(hidden_size_2, 1)
        
        self.relu = nn.LeakyReLU()
        # self.relu = nn.ReLU()
        # self.relu = F.relu
        # self.tanh = nn.Tanh()

    def forward(self, state: Data):
        embedding = self.graph_encoder(state)
        embedding = torch.sum(embedding, dim=0)
        value = self.relu(self.linear1(embedding))
        value = self.relu(self.linear2(value))
        value = self.linear3(value)
        return value


#### A2C

In [ ]:
class ActorCritic(nn.Module):
    """ActorCritic Network"""

    def __init__(self, state_dim, hidden_size_1, hidden_size_2, action_dim):
        super(ActorCritic, self).__init__()
        self.actor = ActorNetwork(
            state_dim=state_dim,
            hidden_size_1=hidden_size_1,
            hidden_size_2=hidden_size_2,
            action_dim=action_dim
        )
        self.critic = CriticNetwork(
            state_dim=state_dim,
            hidden_size_1=hidden_size_1,
            hidden_size_2=hidden_size_2
        )
        self.device = torch.device(
            'cuda' if torch.cuda.is_available() else 'cpu')

    def forward(self, state: Data, jitter=1e-20) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Forward pass, computes action and value
        
        Parameters
        ----------
        state : Data
            Graph state
        jitter : float, optional
            Jitter value, by default 1e-20
        
        Returns
        -------
        Tuple[torch.Tensor, torch.Tensor]
            Tuple of concentration and value
        """
        state = state.to(self.device)
        # Actor
        probs = self.actor(state)
        # Adds jitter to ensure numerical stability
        # Use softplus to ensure concentration is positive
        concentration = F.softplus(probs).reshape(-1) + jitter
        # Critic
        value = self.critic(state)
        return concentration, value
    

### Agent

In [ ]:
class Agent:
    def __init__(
        self, 
        state_dim: int = HyperParams.STATE_DIM.value, 
        hidden_size_1: int = HyperParams.HIDDEN_SIZE_1.value, 
        hidden_size_2: int = HyperParams.HIDDEN_SIZE_2.value,
        action_dim: int = HyperParams.ACTION_DIM.value,
        lr: float = HyperParams.LR.value,
        gamma: float = HyperParams.GAMMA.value,
        eps: float = HyperParams.EPS_CLIP.value,
        best_reward: float = HyperParams.BEST_REWARD.value):
        """
        Initialize the agent.

        Parameters
        ----------
        state_dim : int
            Dimensions of the state, i.e. length of the feature vector
        hidden_size_1 : int
            First A2C hidden layer size
        hidden_size_2 : int
            Second A2C hidden layer size
        action_dim : int
            Dimensions of the action (it is set to 1, to return a tensor N*1)
        lr : float
            Learning rate
        gamma : float
            Discount factor
        eps : float
            Value for clipping the loss function
        best_reward : float, optional
            Best reward, by default 0.8
        """
        self.state_dim = state_dim
        self.hidden_size_1 = hidden_size_1
        self.hidden_size_2 = hidden_size_2
        self.action_dim = action_dim
        self.policy = ActorCritic(
            state_dim, hidden_size_1, hidden_size_2, action_dim)
        
        # Hyperparameters
        self.lr = lr
        self.gamma = gamma
        self.eps = eps
        self.best_reward = best_reward
        # Print Hyperparameters on console
        self.print_hyperparams()
        
        # Set device
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.policy.to(self.device)
        # Set optimizers
        self.optimizers = self.configure_optimizers()
        
        # Initialize lists for logging, it contains: avg_reward, avg_steps per episode
        self.log_dict = HyperParams.LOG_DICT.value
        
        # Training variables
        self.obs = None 
        self.episode_reward = 0
        self.done = False
        self.step = 0
        # action & reward buffer
        self.SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])
        self.saved_actions = []
        self.rewards = []
        
    def configure_optimizers(self):
        """
        Configure optimizers
        
        Returns
        -------
        optimizers : dict
            Dictionary of optimizers
        """
        actor_params = list(self.policy.actor.parameters())
        critic_params = list(self.policy.critic.parameters())
        optimizers = dict()
        optimizers['a_optimizer'] = torch.optim.Adam(actor_params, lr=self.lr)
        optimizers['c_optimizer'] = torch.optim.Adam(critic_params, lr=self.lr)
        return optimizers
    
    def training(
        self,
        env: GraphEnvironment,
        env_name: str,
        detection_alg: str) -> dict:
        """
        Train the agent on the environment, change the target node every 10
        episodes and the target community every 100 episodes. The episode ends
        when the target node is isolated from the target community, or when the
        maximum number of steps is reached.

        Parameters
        ----------
        env : GraphEnvironment
            Environment to train the agent on
        agent : Agent
            Agent to train
        env_name : str
            Name of the environment
        detection_alg : str
            Name of the detection algorithm
        
        Returns
        -------
        log_dict : dict
            Dictionary containing the training logs
        """
        epochs = trange(self.log_dict['train_episodes'])  # epoch iterator
        self.policy.train()  # set model in train mode
        for i_episode in epochs:
            # Change Target Node every 10 episodes
            if i_episode % 10 == 0:
                env.change_target_node()
            # Change Target Community every 100 episodes
            if i_episode % 100 == 0:
                env.change_target_community()
            
            # Rewiring the graph until the target node is isolated from the 
            # target community
            while not self.done:
                self.rewiring(env)
                
            # perform on-policy backpropagation
            self.a_loss, self.v_loss = self.training_step()
            
            # Send current statistics to screen
            epochs.set_description(
                f"Episode {i_episode+1} " +\
                f"| Avg Reward: {self.episode_reward/self.step:.2f} " +\
                f"| Avg Steps: {self.step} " +\
                f"| Actor Loss: {self.a_loss:.2f} " +\
                f"| Critic Loss: {self.v_loss:.2f}")

            # Checkpoint best performing model
            if self.episode_reward >= self.best_reward:
                self.save_checkpoint(env_name, detection_alg)
                self.best_reward = self.episode_reward
            
            # Log
            self.log_dict['train_reward'].append(self.episode_reward)
            self.log_dict['train_steps'].append(self.step)
            self.log_dict['train_avg_reward'].append(
                self.episode_reward/self.step)
            self.log_dict['a_loss'].append(self.a_loss)
            self.log_dict['v_loss'].append(self.v_loss)
            self.log(self.log_dict, env_name, detection_alg)
        return self.log_dict
    
    def rewiring(self, env: GraphEnvironment)->None:
        """
        Rewiring step, select action and take step in environment.

        Parameters
        ----------
        env : GraphEnvironment
            Graph environment
        """
        # Select action: return a list of the probabilities of each action
        action_rl = self.select_action(self.obs)
        torch.cuda.empty_cache()
        # Take action in environment
        self.obs, reward, self.done = env.step(action_rl)
        # Update reward
        self.episode_reward += reward
        # Store the transition in memory
        self.rewards.append(reward)
        self.step += 1
    
    def select_action(self, state: Data) -> int:
        """
        Select action, given a state, using the policy network.
        
        Parameters
        ----------
        state : Data
            Graph state
        
        Returns
        -------
        action: int
            Integer representing a node in the graph, it will be the destination
            node of the rewiring action
        """
        concentration, value = self.policy.forward(state)
        dist = torch.distributions.Categorical(concentration)
        action = dist.sample()
        self.saved_actions.append(
            self.SavedAction(dist.log_prob(action), value))
        return action.item()
    
    def training_step(self) -> Tuple[float, float]:
        """
        Perform a single training step of the A2C algorithm, which involves
        computing the actor and critic losses, taking gradient steps, and 
        resetting the rewards and action buffer.
        
        Returns
        -------
        mean_a_loss : float
            Mean actor loss
        mean_v_loss : float
            Mean critic loss
        """
        R = 0
        saved_actions = self.saved_actions
        policy_losses = []  # list to save actor (policy) loss
        value_losses = []  # list to save critic (value) loss
        returns = []  # list to save the true values

        # calculate the true value using rewards returned from the environment
        for r in self.rewards[::-1]:
            # calculate the discounted value
            R = r + self.gamma * R
            # insert to the beginning of the list
            returns.insert(0, R)

        # Normalize returns by subtracting mean and dividing by standard deviation
        # NOTE: May cause NaN problem
        if len(returns) > 1:
            returns = torch.tensor(returns)
            returns = (returns - returns.mean()) / (returns.std() + self.eps)
        else:
            returns = torch.tensor(returns)

        # Computing losses
        for (log_prob, value), R in zip(saved_actions, returns):
            # Difference between true value and estimated value from critic
            advantage = R - value.item()
            # calculate actor (policy) loss
            policy_losses.append(-log_prob * advantage)
            # calculate critic (value) loss using L1 smooth loss
            value_losses.append(F.smooth_l1_loss(
                value, torch.tensor([R]).to(self.device)))

        # take gradient steps
        self.optimizers['a_optimizer'].zero_grad()
        a_loss = torch.stack(policy_losses).sum()
        a_loss.backward()
        self.optimizers['a_optimizer'].step()

        self.optimizers['c_optimizer'].zero_grad()
        v_loss = torch.stack(value_losses).sum()
        v_loss.backward()
        self.optimizers['c_optimizer'].step()

        mean_a_loss = torch.stack(policy_losses).mean().item()
        mean_v_loss = torch.stack(value_losses).mean().item()

        # reset rewards and action buffer
        del self.rewards[:]
        del self.saved_actions[:]
        return mean_a_loss, mean_v_loss

    def print_hyperparams(self):
        """Print hyperparameters"""
        print("*", "-"*18, "Hyperparameters", "-"*18)
        print("* State dimension: ", self.state_dim)
        print("* Action dimension: ", self.action_dim)
        print("* Learning rate: ", self.lr)
        print("* Gamma parameter: ", self.gamma)
        print("* Value for clipping the loss function: ", self.eps)

    def save_checkpoint(
            self,
            env_name: str = 'default',
            detection_alg: str = 'default',
            log_dir: str = FilePaths.LOG_DIR.value):
        """Save checkpoint"""
        log_dir = log_dir + env_name  # + '/' + detection_alg
        # Check if the directory exists, otherwise create it
        Utils.check_dir(log_dir)
        path = f'{log_dir}/{env_name}_{detection_alg}.pth'
        checkpoint = dict()
        checkpoint['model'] = self.policy.state_dict()
        for key, value in self.optimizers.items():
            checkpoint[key] = value.state_dict()
        torch.save(checkpoint, path)

    def load_checkpoint(
            self,
            env_name: str = 'default',
            detection_alg: str = 'default',
            log_dir: str = FilePaths.LOG_DIR.value):
        """Load checkpoint
        
        Parameters
        ----------
        env_name : str, optional
            Environment name, by default 'default'
        detection_alg : str, optional
            Detection algorithm name, by default 'default'
        log_dir : str, optional
            Path to the log directory, by default FilePaths.LOG_DIR.value
        """
        log_dir = log_dir + env_name  # + '/' + detection_alg
        path = f'{log_dir}/{env_name}_{detection_alg}.pth'
        checkpoint = torch.load(path)
        self.policy.load_state_dict(checkpoint['model'])
        for key, _ in self.optimizers.items():
            self.optimizers[key].load_state_dict(checkpoint[key])

    def log(
            self,
            log_dict: dict,
            env_name: str = 'default',
            detection_alg: str = 'default',
            log_dir: str = FilePaths.LOG_DIR.value):
        """Log data
        
        Parameters
        ----------
        log_dict : dict
            Dictionary containing the data to be logged
        env_name : str, optional
            Environment name, by default 'default'
        detection_alg : str, optional
            Detection algorithm name, by default 'default'
        log_dir : str, optional
            Path to the log directory, by default FilePaths.LOG_DIR.value
        """
        log_dir = log_dir + env_name  # + '/' + detection_alg
        path = f'{log_dir}/{env_name}_{detection_alg}.pth'
        torch.save(log_dict, path)


## Execution

In [ ]:
print("*"*20, "Setup Information", "*"*20)

# ° ------ Graph Setup ------ ° #
# ! REAL GRAPH Graph path (change the following line to change the graph)
graph_path = FilePaths.KAR.value
# Load the graph from the dataset folder
graph = Utils.import_mtx_graph(graph_path)
# ! SYNTHETIC GRAPH Graph path (change the following line to change the graph)
# graph, graph_path = Utils.generate_lfr_benchmark_graph()

# Set the environment name as the graph name
env_name = graph_path.split("/")[-1].split(".")[0]

# Print the number of nodes and edges
print("* Graph Name:", env_name)
print("*", graph)

# ° --- Environment Setup --- ° #
# ! Define the detection algorithm to use (change the following line to change the algorithm)
detection_alg = DetectionAlgorithms.WALK.value
print("* Community Detection Algorithm:", detection_alg)
# Apply the community detection algorithm on the graph
dct = CommunityDetectionAlgorithm(detection_alg)
community_structure = dct.compute_community(graph)
print("* Number of communities found:", len(community_structure.communities))

# Choose one of the communities found by the algorithm, for now we choose 
# the community with the highest number of nodes
community_target = max(community_structure.communities, key=len)
idx_community = community_structure.communities.index(community_target)
print("* Community Target:\t", community_target)
print("* Index Community:\t", idx_community)
# TEST: Choose a node to remove from the community
node_target = community_target[random.randint(0, len(community_target)-1)]
print("* Nodes Target:\t\t", node_target)

# Define the environment
env = GraphEnvironment(
graph=graph,
community=community_target,
idx_community=idx_community,
node_target=node_target,
env_name=env_name,
community_detection_algorithm=detection_alg)
# Get list of possible actions which can be performed on the graph by the agent
n_actions = len(env.possible_actions["ADD"]) + \
len(env.possible_actions["REMOVE"])
print("* Number of possible actions:", n_actions)
print("* Rewiring Budget:", env.edge_budget)

# ° ------ Agent Setup ------ ° #
# Define the agent
agent = Agent()
# Print Hyperparameters of the Agent (inner method)
print("*", "-"*53)
print("*"*20, "End Information", "*"*20, "\n")

log = agent.training(env, env_name, detection_alg)
file_path = FilePaths.TEST_DIR.value + env_name + '/' + detection_alg
Utils.check_dir(file_path)
Utils.save_training(log, env_name, detection_alg, file_path=file_path)
Utils.plot_training(log, env_name, detection_alg, file_path=file_path)